## Tree Based Ensemble Models for Price Returns Forecasting

### Approach
1. Set Up
2. Identifying Suitable Lags for Price, Volume & Uncertainty Indices
3. Modelling w/ Grid Search & Forecast Evaluation
   1. Random Forest (h = 1 to 12)
      1. Model A (With Price Returns, Price & Volume)
      2. Model B (With A + Lucey Original Price) 
      3. Model C (With A + Lucey Reddit Price)
      4. Model D (With A + LDA Reddit Price)
      5. Model E (With A + Top2Vec Reddit Price)
      6. Model F (With A + VCRIX)
      7. Policy Based Models?

TBD:
1. Add in More Horizon Values [Running]
2. Compute historical forecasts with best models => Direction of Forecast

### Set Up

In [66]:
# NB config
%load_ext autoreload
%autoreload 2

# Load Libraries
import os

os.chdir("../../")
from typing import Any, List, Dict, Union
import pandas as pd
import numpy as np
from pathlib import Path
from darts import TimeSeries
from darts.metrics import (
    rmse,
)
from tqdm import tqdm
from darts import concatenate
from darts.models.forecasting.random_forest import (
    RandomForest,
)
from sklearn.model_selection import ParameterGrid
import warnings

warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Preparation

In [2]:
# Data Dir
data_dir = Path("forecasting/data/modelling")

# BTC-USD data
btc_usd_fp = data_dir / "btc_usd_weekly.csv"
btc_usd_df = pd.read_csv(btc_usd_fp)

# UCRY Indices data
ucry_fp = data_dir / "ucry_indices_weekly.csv"
ucry_df = pd.read_csv(ucry_fp)

#### Create ***h***-weeks Log Price Returns Time Series

In [3]:
# h = 1 (Weekly Price Returns)
btc_usd_df["Price Returns (h=1)"] = np.log1p(btc_usd_df[["Price"]].pct_change(1))

# h = 4 (4 Week Price Returns)
btc_usd_df["Price Returns (h=4)"] = np.log1p(btc_usd_df[["Price"]].pct_change(4))

# h = 12 (12 Week Price Returns)
btc_usd_df["Price Returns (h=12)"] = np.log1p(btc_usd_df[["Price"]].pct_change(12))

# Create TimeSeries
# h = 1 (Weekly Price Returns)
btc_usd1_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=1)"]].dropna(), time_col="Date"
)

# h = 4 (4 Week Price Returns)
btc_usd4_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=4)"]].dropna(), time_col="Date"
)

# h = 12 (12 Week Price Returns)
btc_usd12_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=12)"]].dropna(), time_col="Date"
)

In [4]:
def gen_log_price_returns(
    series: pd.DataFrame, h: int, date_col: str = "Date", var_col: str = "Price"
) -> TimeSeries:
    new_col_name = f"Price Returns (h={h})"
    series[new_col_name] = np.log1p(series[[var_col]].pct_change(h))
    new_ts = TimeSeries.from_dataframe(
        series[[date_col, new_col_name]].dropna(), time_col=date_col
    )
    return new_ts

### Create Price and Volume Time Series

In [5]:
# Price
price_ts = TimeSeries.from_dataframe(btc_usd_df[["Date", "Price"]], time_col="Date")

# Volume
vol_ts = TimeSeries.from_dataframe(btc_usd_df[["Date", "Volume"]], time_col="Date")

#### Create UCRY Indices Time Series

In [6]:
# Create TimeSeries
sel_cols = ["Date", "Index Value"]
time_col = "Date"

# Lucey Price
lucey_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Lucey-Original-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# Lucey Reddit Price
lucey_reddit_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Lucey-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# LDA Price
lda_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "LDA-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# Top2Vec Price
t2v_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Top2Vec-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# VCRIX
vcrix = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "VCRIX"].reset_index()[sel_cols], time_col=time_col
)

#### Train Test Split Date

In [7]:
# Split into Train and Test
split_date = pd.Timestamp("20190527")

### Identify Suitable Lags for UCRY Index Time Series
* STATUS: Use **t** variables to predict **t + h** variables for now

### Random Forest Forecasting Model & Evaluation

#### Define Params Grid for Grid Search

In [8]:
# Params Grid
rf_params_grid = {
    "n_estimators": [50, 100, 300],
    "max_depth": [2, 5, 10],
    "criterion": ["squared_error"],
    "max_features": [1 / 3, "auto"],
    "n_jobs": [-1],
    "random_state": [42],
    "oob_score": [True],
}

rf_params_list = list(ParameterGrid(rf_params_grid))
len(rf_params_list)

18

### Define Forecast Horizons

In [9]:
horizons = list(range(1, 13))

### Random Forest GridSearch Helper

In [10]:
# Run Grid Search (Runs for super long may be buggy - Revisit in the future)
# rfA_1_best_model, rfA_1_best_params = RandomForest(
#    lags=1,
#     lags_past_covariates=1
# ).gridsearch(
#     parameters=rf_params_grid,
#     series=btc_usd1_ts,
#     past_covariates=rfA_1_past_covs,
#     forecast_horizon=1,
#     stride=1,
#     start=split_date,
#     metric=mse,
#     reduction=np.mean,
#     verbose=True,
#     n_jobs=-1
# )
#
# pprint(rfA_1_best_params)

In [11]:
# Homemade RF Grid Search


def gridsearch_RF(
    series: TimeSeries,
    past_covariates: TimeSeries,
    forecast_horizon: int,
    lags: int,
    lags_past_covariates: int,
    verbose: bool = False,
    params_list: List[Any] = rf_params_list,
    error_metric: Any = rmse,
    split_date: pd.Timestamp = split_date,
):

    min_error = np.float("inf")
    best_params = None
    for params in tqdm(params_list, leave=True):
        model = RandomForest(
            lags=lags, lags_past_covariates=lags_past_covariates, **params
        )
        error = model.backtest(
            series=series,
            past_covariates=past_covariates,
            forecast_horizon=forecast_horizon,
            stride=1,
            start=split_date,
            metric=error_metric,
            reduction=np.mean,
            verbose=verbose,
        )
        if error < min_error:
            min_error = error
            best_params = params

    print("Average RMSE over all historical forecasts: %.2f" % min_error)
    print("Best Params: %s" % str(best_params))

    return best_params, min_error

In [12]:
# Multi Horizon Forecasting Helper


def multi_forecast_GS_RF(
    target_df: pd.DataFrame,
    past_covariates: List[TimeSeries],
    forecast_horizons: List[int] = horizons,
    lags: int = 1,
    past_covariates_lags: int = 1,
    verbose: bool = False,
    error_metric: Any = rmse,
    params_list: List[Any] = rf_params_list,
    split_date: pd.Timestamp = split_date,
) -> None:

    best_results = {}

    for h in tqdm(forecast_horizons):
        print(f"Recursively Forecasting horizon={h} with Random Forest + Grid Search")

        # Generate Appropriate TS Data
        target_cov = gen_log_price_returns(target_df, h)
        past_cov_list = [target_cov]
        (
            past_cov_list.extend(
                list(map(lambda x: x.slice_intersect(target_cov), past_covariates))
            )
        )
        past_cov_list_tidy = concatenate(past_cov_list, axis=1)
        # Run Grid Search RF
        best_params, min_error = gridsearch_RF(
            target_cov,
            past_cov_list_tidy,
            h,
            lags,
            past_covariates_lags,
            verbose,
            params_list,
            error_metric,
            split_date,
        )

        best_results[h] = {"best_params": best_params, "min_error": min_error}

    return best_results

#### Model A (Price Returns, Price & Volume) for h = 1 to 12

In [13]:
# Model A Multi Horizon
RF_A_results = multi_forecast_GS_RF(
    btc_usd_df,
    [price_ts, vol_ts],
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [05:22<59:04, 322.26s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 0.3333333333333333, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [11:12<56:29, 339.00s/it]

...

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 100, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:22:55<08:21, 501.26s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:31:56<00:00, 459.68s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


#### Model B (Price Returns, Price & Volume + Lucey Price Index)

In [14]:
# Model B Multi Horizon
RF_B_results = multi_forecast_GS_RF(
    btc_usd_df,
    [price_ts, vol_ts, lucey_price],
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [06:30<1:11:34, 390.42s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 0.3333333333333333, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [13:19<1:06:54, 401.45s/it]

...

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:24:56<08:21, 501.51s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:33:56<00:00, 469.72s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


#### Model C (Price Returns, Price & Volume + Lucey Reddit Price Index)

In [15]:
# Model C Multi Horizon
RF_C_results = multi_forecast_GS_RF(
    btc_usd_df,
    [price_ts, vol_ts, lucey_reddit_price],
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [06:30<1:11:40, 390.92s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 0.3333333333333333, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [13:19<1:06:53, 401.38s/it]

...

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:24:58<08:21, 501.94s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:33:59<00:00, 469.92s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


#### Model D (Price Returns, Price & Volume + LDA Reddit Price Index)

In [16]:
# Model D Multi Horizon
RF_D_results = multi_forecast_GS_RF(
    btc_usd_df,
    [price_ts, vol_ts, lda_price],
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [06:30<1:11:37, 390.68s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 0.3333333333333333, 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [13:19<1:06:52, 401.21s/it]

...

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:24:55<08:21, 501.51s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:33:55<00:00, 469.65s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


#### Model E (Price Returns, Price & Volume + Top2Vec Reddit Price Index)

In [21]:
# Model E Multi Horizon
RF_E_results = multi_forecast_GS_RF(
    btc_usd_df,
    [price_ts, vol_ts, t2v_price],
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [06:31<1:11:43, 391.25s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 0.3333333333333333, 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [13:20<1:06:59, 401.98s/it]

...

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:25:10<08:23, 503.17s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:34:12<00:00, 471.02s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


#### Model F (Price Returns, Price & Volume + VCRIX) [TBD: Special Case due to cov concat]

In [19]:
def multi_forecast_GS_RF_VCRIX(
    target_df: pd.DataFrame,
    forecast_horizons: List[int] = horizons,
    lags: int = 1,
    past_covariates_lags: int = 1,
    verbose: bool = False,
    error_metric: Any = rmse,
    params_list: List[Any] = rf_params_list,
    split_date: pd.Timestamp = split_date,
) -> None:

    best_results = {}

    for h in tqdm(forecast_horizons):
        print(f"Recursively Forecasting horizon={h} with Random Forest + Grid Search")

        # Generate Appropriate TS Data
        target_cov = gen_log_price_returns(target_df, h)
        past_cov_list = concatenate(
            [
                target_cov.slice_intersect(vcrix),
                price_ts.slice_intersect(vcrix),
                vol_ts.slice_intersect(vcrix),
                vcrix.slice_intersect(target_cov.slice_intersect(vcrix)),
            ],
            axis=1,
        )
        # Run Grid Search RF
        best_params, min_error = gridsearch_RF(
            target_cov,
            past_cov_list,
            h,
            lags,
            past_covariates_lags,
            verbose,
            params_list,
            error_metric,
            split_date,
        )

        best_results[h] = {"best_params": best_params, "min_error": min_error}

    return best_results

In [20]:
RF_F_results = multi_forecast_GS_RF_VCRIX(btc_usd_df)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest + Grid Search


  8%|▊         | 1/12 [05:18<58:28, 318.94s/it]

Average RMSE over all historical forecasts: 0.08
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 0.3333333333333333, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=2 with Random Forest + Grid Search


 17%|█▋        | 2/12 [10:52<54:33, 327.34s/it]

...

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=11 with Random Forest + Grid Search


 92%|█████████▏| 11/12 [1:19:41<08:14, 494.96s/it]

Average RMSE over all historical forecasts: 0.18
Best Params: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}
Recursively Forecasting horizon=12 with Random Forest + Grid Search


100%|██████████| 12/12 [1:28:39<00:00, 443.32s/it]

Average RMSE over all historical forecasts: 0.17
Best Params: {'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 300, 'n_jobs': -1, 'oob_score': True, 'random_state': 42}


### Consolidate and Save Best Model Results [Add to end of each GridSearch later]


In [50]:
save_data_dir = Path(
    "/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/forecasting/data/forecasts/random_forest"
)

In [51]:
def format_results(res_dict: Dict[int, Any], model_name: str) -> pd.DataFrame:
    df = pd.DataFrame(res_dict).T.reset_index().rename(columns={"index": "horizon"})
    df["model"] = model_name
    return df

In [52]:
RF_A_results_df = format_results(RF_A_results, "A")
RF_B_results_df = format_results(RF_B_results, "B")
RF_C_results_df = format_results(RF_C_results, "C")
RF_D_results_df = format_results(RF_D_results, "D")
RF_E_results_df = format_results(RF_E_results, "E")
RF_F_results_df = format_results(RF_F_results, "F")

In [53]:
# Save as csv
models = ["A", "B", "C", "D", "E", "F"]

res = [
    RF_A_results_df,
    RF_B_results_df,
    RF_C_results_df,
    RF_D_results_df,
    RF_E_results_df,
    RF_F_results_df,
]

model_res_collection = zip(models, res)

for name, res in model_res_collection:
    path = save_data_dir / f"model_{name}_results.csv"
    res.to_csv(path, index=False)

In [61]:
RF_A_results_df.query("horizon == 1").best_params.iloc[0]

{'criterion': 'squared_error',
 'max_depth': 2,
 'max_features': 0.3333333333333333,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': True,
 'random_state': 42}

### Historical Forecasts with Best Model

In [88]:
hist_forecast_save_dir = Path(
    "/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/forecasting/data/forecasts/random_forest"
)

In [89]:
# Homemade RF Grid Search


def hist_forecast_RF(
    series: TimeSeries,
    past_covariates: TimeSeries,
    forecast_horizon: int,
    best_params: Dict[str, Any],
    lags: int,
    lags_past_covariates: int,
    verbose: bool = False,
    split_date: pd.Timestamp = split_date,
):

    model = RandomForest(
        lags=lags, lags_past_covariates=lags_past_covariates, **best_params
    )
    hist_forecast = model.historical_forecasts(
        series=series,
        past_covariates=past_covariates,
        forecast_horizon=forecast_horizon,
        stride=1,
        start=split_date,
        verbose=verbose,
    )
    return hist_forecast

In [93]:
def multi_hist_forecast_RF(
    target_df: pd.DataFrame,
    past_covariates: List[TimeSeries],
    best_params_dict: Dict[int, Any],
    save_data_dir: Union[str, Path],
    model_name: str,
    lags: int = 1,
    past_covariates_lags: int = 1,
    verbose: bool = True,
    split_date: pd.Timestamp = split_date,
) -> None:

    for h in tqdm(best_params_dict.keys()):
        print(f"Recursively Forecasting horizon={h} with Random Forest")

        # Generate Appropriate TS Data
        target_cov = gen_log_price_returns(target_df, h)
        past_cov_list = [target_cov]
        (
            past_cov_list.extend(
                list(map(lambda x: x.slice_intersect(target_cov), past_covariates))
            )
        )
        past_cov_list_tidy = concatenate(past_cov_list, axis=1)
        # Run Historical Forecast
        hist_forecast = hist_forecast_RF(
            target_cov,
            past_cov_list_tidy,
            h,
            best_params_dict[h],
            lags,
            past_covariates_lags,
            verbose,
            split_date,
        )
        print(f"Saving forecasts ...")
        forecast_df = hist_forecast.with_columns_renamed(["0"], ["price_return"])
        forecast_df.to_csv(Path(save_data_dir) / f"rf_model_{model_name}_h{h}.csv")

### Model A

In [94]:
RF_A_best_params_dict = RF_A_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [95]:
multi_hist_forecast_RF(
    btc_usd_df, [price_ts, vol_ts], RF_A_best_params_dict, hist_forecast_save_dir, "A"
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:11<02:01, 11.03s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [02:23<00:13, 13.46s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [02:33<00:00, 12.83s/it]

Saving forecasts ...


### Model B

In [96]:
RF_B_best_params_dict = RF_B_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [97]:
multi_hist_forecast_RF(
    btc_usd_df,
    [price_ts, vol_ts, lucey_price],
    RF_B_best_params_dict,
    hist_forecast_save_dir,
    "B",
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:30<05:34, 30.37s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [05:48<00:30, 30.16s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [06:33<00:00, 32.80s/it]

Saving forecasts ...


### Model C

In [98]:
RF_C_best_params_dict = RF_C_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [99]:
multi_hist_forecast_RF(
    btc_usd_df,
    [price_ts, vol_ts, lucey_reddit_price],
    RF_C_best_params_dict,
    hist_forecast_save_dir,
    "C",
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:11<02:08, 11.66s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [05:29<00:29, 29.96s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [06:14<00:00, 31.20s/it]

Saving forecasts ...


### Model D

In [100]:
RF_D_best_params_dict = RF_D_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [101]:
multi_hist_forecast_RF(
    btc_usd_df,
    [price_ts, vol_ts, lda_price],
    RF_D_best_params_dict,
    hist_forecast_save_dir,
    "D",
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:06<01:08,  6.24s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [05:25<00:29, 29.99s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [06:10<00:00, 30.88s/it]

Saving forecasts ...


### Model E

In [102]:
RF_E_best_params_dict = RF_E_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [103]:
multi_hist_forecast_RF(
    btc_usd_df,
    [price_ts, vol_ts, t2v_price],
    RF_E_best_params_dict,
    hist_forecast_save_dir,
    "E",
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:06<01:06,  6.09s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [05:26<00:30, 30.12s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [06:13<00:00, 31.16s/it]

Saving forecasts ...


### Model F

In [110]:
def multi_hist_forecast_RF_VCRIX(
    target_df: pd.DataFrame,
    best_params_dict: Dict[int, Any],
    save_data_dir: Union[str, Path],
    model_name: str,
    lags: int = 1,
    past_covariates_lags: int = 1,
    verbose: bool = True,
    split_date: pd.Timestamp = split_date,
) -> None:

    for h in tqdm(best_params_dict.keys()):
        print(f"Recursively Forecasting horizon={h} with Random Forest")

        # Generate Appropriate TS Data
        target_cov = gen_log_price_returns(target_df, h)
        past_cov_list = concatenate(
            [
                target_cov.slice_intersect(vcrix),
                price_ts.slice_intersect(vcrix),
                vol_ts.slice_intersect(vcrix),
                vcrix.slice_intersect(target_cov.slice_intersect(vcrix)),
            ],
            axis=1,
        )

        # Run Historical Forecast
        hist_forecast = hist_forecast_RF(
            target_cov,
            past_cov_list,
            h,
            best_params_dict[h],
            lags,
            past_covariates_lags,
            verbose,
            split_date,
        )
        print(f"Saving forecasts ...")
        forecast_df = hist_forecast.with_columns_renamed(["0"], ["price_return"])
        forecast_df.to_csv(Path(save_data_dir) / f"rf_model_{model_name}_h{h}.csv")

In [111]:
RF_F_best_params_dict = RF_F_results_df.set_index("horizon")[["best_params"]].to_dict()[
    "best_params"
]

In [112]:
multi_hist_forecast_RF_VCRIX(
    btc_usd_df, RF_F_best_params_dict, hist_forecast_save_dir, "F"
)

  0%|          | 0/12 [00:00<?, ?it/s]

Recursively Forecasting horizon=1 with Random Forest


  0%|          | 0/136 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:10<01:57, 10.69s/it]

Saving forecasts ...
Recursively Forecasting horizon=2 with Random Forest


...

 92%|█████████▏| 11/12 [06:12<00:42, 42.25s/it]

Saving forecasts ...
Recursively Forecasting horizon=12 with Random Forest


  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 12/12 [06:58<00:00, 34.86s/it]

Saving forecasts ...
